**stage3_model_eval**. This notebook evaluates the trained model

**Edit**<br/>

**TODO**<br/>

# Import packages and get authenticated

In [ ]:
# from google.colab import drive
# drive.mount('drive')

In [1]:
import numpy as np
import copy
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = "{:,.6f}".format

from tqdm import tqdm_notebook as tqdm
from IPython.display import display
from PIL import Image
import os
import sys
sys.path.append('/content/drive/My Drive/中研院/repo/')
# sys.path.append('~/project_FDDAT/repo/')
sys.path.append('../') # add this line so Data and data are visible in this file
from os.path import expanduser
home = expanduser("~")

from falldetect.utilities import *
from falldetect.models import *
from falldetect.dataset_util import *
from falldetect.training_util import *
from falldetect.eval_util import *

import time
import datetime
from datetime import datetime
import json
import argparse

# Plotting
# checklist 1: comment inline, uncomment Agg
%matplotlib inline
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib import pyplot
matplotlib.rc( 'savefig', facecolor = 'white' )

from sklearn.decomposition import PCA

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

# Get user inputs
In ipython notebook, these are hardcoded. In production python code, use parsers to provide these inputs

In [64]:
parser = argparse.ArgumentParser(description='FD_DAT')
parser.add_argument('--input_folder', metavar='input_folder', help='input_folder',
                    default='../')
parser.add_argument('--output_folder', metavar='output_folder', help='output_folder',
                    default='../')
parser.add_argument('--training_params_file', metavar='training_params_file', help='training_params_file',
                    default='training_params_list.json')
parser.add_argument('--tasks_list', metavar='tasks_list', help='a list of all tasks',
                    default='UMAFall_waist_UPFall_belt UPFall_wrist_UMAFall_ankle')
parser.add_argument('--variable_name', metavar='variable_name', help='key in training_params to be displayed on plot',
                    default='HP_name')
parser.add_argument('--debug_F1', metavar='debug_F1', help='debug F1',
                    default='False')



# parser.add_argument('--src_names', metavar='src_names', help='a list of src_names',
#                     default='UMAFall_waist_UPFall_belt UPFall_wrist_UMAFall_ankle')
# parser.add_argument('--tgt_names', metavar='tgt_names', help='a list of tgt_names',
#                     default='UMAFall_waist_UPFall_belt UPFall_wrist_UMAFall_ankle')

# checklist 2: comment first line, uncomment second line seizures_FN

# args = parser.parse_args(['--input_folder', '../../data_mic/stage2/modeloutput_18hz_5fold_UPFall_UMAFall_cross-config_diffCV',
#                           '--output_folder', '../../data_mic/stage3/test',


# args = parser.parse_args(['--input_folder', '../../data_mic/stage2/modeloutput_WithoutNormal_18hz_5fold_UPFall_UMAFall_cross-config_diffCV',
# args = parser.parse_args(['--input_folder', '../../data_mic/stage2/modeloutput_18hz_5fold_UPFall_UMAFall_cross-config_diffCV_earlystop',
# args = parser.parse_args(['--input_folder', '../../data_mic/stage2/test',
# args = parser.parse_args(['--input_folder', '../../data_mic/stage2/modeloutput_18hz_5fold_UPFall_UMAFall_cross-config_HPsearch',
#                           '--output_folder', '../../data_mic/stage3/test',
# #                           '--training_params_file', 'training_params_list_fixed.json',
#                           '--training_params_file', 'training_params_list_HPsearch.json',
# #                           '--tasks_list', 'UMAFall_chest-UPFall_neck UMAFall_wrist-UPFall_wrist UMAFall_waist-UPFall_belt UMAFall_leg-UPFall_rightpocket UMAFall_ankle-UPFall_ankle',
#                           '--tasks_list', 'UMAFall_chest-UPFall_neck UMAFall_leg-UPFall_rightpocket',
#                           '--variable_name', 'HP_name',
#                           '--debug_F1', 'True',])

args = parser.parse_args(['--input_folder', '../../data_mic/stage2/modeloutput_18hz_5fold_UMAFall_cross-pos_loss_earlystop_revision',
                          '--output_folder', '../../data_mic/stage3/UMAFall_cross-pos_loss_earlystop_revision',
#                           '--training_params_file', 'training_params_list_fixed.json',
                          '--training_params_file', 'training_params_list_fixed_revision.json',
#                           '--tasks_list', 'UMAFall_chest-UPFall_neck UMAFall_wrist-UPFall_wrist UMAFall_waist-UPFall_belt UMAFall_leg-UPFall_rightpocket UMAFall_ankle-UPFall_ankle',
                          '--tasks_list', 'UMAFall_chest-UMAFall_waist UMAFall_chest-UMAFall_wrist UMAFall_chest-UMAFall_ankle UMAFall_waist-UMAFall_wrist UMAFall_waist-UMAFall_ankle UMAFall_waist-UMAFall_chest UMAFall_wrist-UMAFall_ankle UMAFall_wrist-UMAFall_waist UMAFall_wrist-UMAFall_chest UMAFall_ankle-UMAFall_chest UMAFall_ankle-UMAFall_waist UMAFall_ankle-UMAFall_wrist',
                          '--variable_name', 'HP_name',
                          '--debug_F1', 'True',])

#                           '--src_names', 'UPFall_neck UPFall_wrist UPFall_belt UPFall_rightpocket UPFall_ankle',
#                           '--tgt_names', 'UMAFall_chest UMAFall_wrist UMAFall_waist UMAFall_leg UMAFall_ankle'])

# args = parser.parse_args()

In [65]:
home_dir = home+'/project_FDDAT/'
input_folder = args.input_folder
output_folder = args.output_folder
training_params_file = args.training_params_file

tasks_list = []
for item in args.tasks_list.split(' '):
    tasks_list.append((item.split('-')[0], item.split('-')[1]))
    
# src_domains = args.src_names.split(' ')
# tgt_domains = args.tgt_names.split(' ')
variable_name = args.variable_name

if args.debug_F1=='True':
    debug_F1 = True
else:
    debug_F1 = False


inputdir = input_folder + '/'
outputdir = output_folder + '/'
if not os.path.exists(outputdir):
    os.makedirs(outputdir)

In [66]:
outputdir

'../../data_mic/stage3/UMAFall_cross-pos_loss_earlystop_revision/'

In [67]:
with open('../stage2/'+training_params_file) as json_file:
    training_params_list = json.load(json_file)

# TODO: need to fix once training_params_list is fixed
# training_params_list.pop(-2)
training_params_list

[{'HP_name': 'HP_fixed',
  'classes_n': 2,
  'channel_n': 4,
  'batch_size': 4,
  'learning_rate': 0.001,
  'λ': 1,
  'dropout': 0.5,
  'show_diagnosis_plt': True,
  'use_WeightedRandomSampler': True,
  'ADL_only': True}]

In [68]:

# def plot_task_metric(df_temp, metric_name, outputdir):
#     source_means = df_temp.loc['source',df_temp.columns != 'average'].apply(get_mean).values
#     DANN_means = df_temp.loc['DANN',df_temp.columns != 'average'].apply(get_mean).values
#     task_names = df_temp.columns[df_temp.columns != 'average']

#     fig = plt.figure(figsize=(5, 5), dpi=dpi)
#     ax = fig.add_subplot(1, 1, 1)
#     ax.scatter(source_means, DANN_means, s=40, marker='o')
#     ax_xlim = ax.get_xlim()
#     ax_ylim = ax.get_ylim()
#     ax.plot([0, 1], [0, 1], c=".3", linewidth=1, alpha=0.4)
#     ax.set_title('{}'.format(metric_name.split('_')[1]), fontsize=20)
#     ax.set_xlabel('source_means', fontsize=15)
#     ax.set_ylabel('DANN_means', fontsize=15)   # relative to plt.rcParams['font.size']
#     ax.set_xlim(min(ax_xlim[0], ax_ylim[0]), max(ax_xlim[1], ax_ylim[1]))
#     ax.set_ylim(min(ax_xlim[0], ax_ylim[0]), max(ax_xlim[1], ax_ylim[1]))

#     for i, txt in enumerate(task_names):
#         ax.annotate(txt, (source_means[i], DANN_means[i]), fontsize=10, textcoords="offset points", xytext=(0,10), ha='center')
#     fig.savefig(outputdir+'scatter_{}.png'.format(metric_name.split('_')[1]))


In [94]:
df_metric_keys = ['df_acc', 'df_sensitivity', 'df_precision', 'df_F1']
# metric_names = ['acc', 'sensitivity', 'precision', 'F1','PAD']
metric_names = ['sensitivity', 'specificity', 'precision', 'F1','PAD']
training_setting_list = ['source', 'DANN', 'target', 'improvement']

In [95]:
dict_task_name_list = []
for training_params in training_params_list:
#     dict_task_name_list.append('N_ch={}'.format(training_params['channel_n']))
    dict_task_name_list.append(training_params['HP_name'])
    training_params['rep_n'] = 5
    training_params['CV_n'] = 5

dict_task_name_list

['HP_fixed']

In [96]:
print('HP_name\t\tchannel_n')
for training_params in training_params_list:
    print('{}\t\t{}'.format(training_params['HP_name'], training_params['channel_n']))

HP_name		channel_n
HP_fixed		4


In [107]:
df_task_list = dict( zip(df_metric_keys,[[], [], [], []]))
training_types = ['source','dann','target']

dict_task_all = {}
for task_item in tasks_list:
    (src_name, tgt_name) = task_item
    print(task_item)
    
    dict_list = []
    for _ in range(len(dict_task_name_list)):
        dict_list.append({}) 
    
    dict_task = dict( zip( dict_task_name_list,dict_list ) )

    try:
        training_type = 'source'
        for training_params in training_params_list:
            df_list = []
            for i_rep in range(training_params['rep_n']):
                df_inputdir = inputdir+src_name+'_'+tgt_name+'/{}/{}/rep{}/df_performance.csv'.format(training_params['HP_name'],training_type,i_rep)
                df = pd.read_csv(df_inputdir, index_col=0).iloc[0:training_params['CV_n']][['val_tgt_acc','val_tgt_sensitivity','val_tgt_precision','val_tgt_F1','PAD','total_loss']]
                df = df.rename(columns={'val_tgt_acc':'acc','val_tgt_sensitivity':'sensitivity','val_tgt_precision':'precision','val_tgt_F1':'F1','total_loss':'loss'})
                df_list.append(df)

            dict_task[training_params['HP_name']]['performance_{}'.format(training_type)] = pd.concat(df_list,ignore_index=True)

        training_type = 'dann'
        for training_params in training_params_list:
            df_list = []
            for i_rep in range(training_params['rep_n']):
                df_inputdir = inputdir+src_name+'_'+tgt_name+'/{}/{}/rep{}/df_performance.csv'.format(training_params['HP_name'],training_type,i_rep)
                df = pd.read_csv(df_inputdir, index_col=0).iloc[0:training_params['CV_n']][['val_tgt_acc','val_tgt_sensitivity','val_tgt_precision','val_tgt_F1','PAD','total_loss']]
                df = df.rename(columns={'val_tgt_acc':'acc','val_tgt_sensitivity':'sensitivity','val_tgt_precision':'precision','val_tgt_F1':'F1','total_loss':'loss'})
                df_list.append(df)

            dict_task[training_params['HP_name']]['performance_{}'.format(training_type)] = pd.concat(df_list,ignore_index=True)

        training_type = 'target'
        for training_params in training_params_list:
            df_list = []
            for i_rep in range(training_params['rep_n']):
                df_inputdir = inputdir+src_name+'_'+tgt_name+'/{}/{}/rep{}/df_performance.csv'.format(training_params['HP_name'],training_type,i_rep)
                df = pd.read_csv(df_inputdir, index_col=0).iloc[0:training_params['CV_n']][['val_src_acc','val_src_sensitivity','val_src_precision','val_src_F1','PAD','total_loss']]
                df = df.rename(columns={'val_src_acc':'acc','val_src_sensitivity':'sensitivity','val_src_precision':'precision','val_src_F1':'F1','total_loss':'loss'})
                df_list.append(df)

            dict_task[training_params['HP_name']]['performance_{}'.format(training_type)] = pd.concat(df_list,ignore_index=True)

        dict_task_all[src_name+'_'+tgt_name] = dict_task
        
    except:
        print("Oops!", sys.exc_info()[0], "occurred.")


('UMAFall_chest', 'UMAFall_waist')
('UMAFall_chest', 'UMAFall_wrist')
('UMAFall_chest', 'UMAFall_ankle')
('UMAFall_waist', 'UMAFall_wrist')
('UMAFall_waist', 'UMAFall_ankle')
('UMAFall_waist', 'UMAFall_chest')
('UMAFall_wrist', 'UMAFall_ankle')
('UMAFall_wrist', 'UMAFall_waist')
('UMAFall_wrist', 'UMAFall_chest')
('UMAFall_ankle', 'UMAFall_chest')
('UMAFall_ankle', 'UMAFall_waist')
('UMAFall_ankle', 'UMAFall_wrist')


In [108]:
df

,acc,sensitivity,precision,F1,PAD,loss
0,0.975000,0.939394,1.000000,0.968750,1.944814,0.000839
1,0.981250,1.000000,0.953846,0.976378,1.996108,0.000327
2,1.000000,1.000000,1.000000,1.000000,1.858093,0.000190
3,1.000000,1.000000,1.000000,1.000000,1.922075,0.000311
4,0.920000,0.828571,1.000000,0.906250,1.957799,0.001585


In [98]:
dict_task_all.keys()

dict_keys(['UMAFall_chest_UMAFall_waist', 'UMAFall_chest_UMAFall_wrist', 'UMAFall_chest_UMAFall_ankle', 'UMAFall_waist_UMAFall_wrist', 'UMAFall_waist_UMAFall_ankle', 'UMAFall_waist_UMAFall_chest', 'UMAFall_wrist_UMAFall_ankle', 'UMAFall_wrist_UMAFall_waist', 'UMAFall_wrist_UMAFall_chest', 'UMAFall_ankle_UMAFall_chest', 'UMAFall_ankle_UMAFall_waist', 'UMAFall_ankle_UMAFall_wrist'])

In [99]:
color_dict = {'Green': '#3cb44b', 
              'Sunglow': '#FFD43B',
              'Red': '#e6194b', 
              'Blue': '#0082c8', 

              'Teal': '#008080', 

              'Maroon': '#800000', 
              'Navy': '#000080', 
              'Mint': '#aaffc3', 
              'Yellow': '#ffe119', 

              'Orange': '#f58231', 
              'Purple': '#911eb4', 
              'Cyan': '#46f0f0', 
              'Magenta': '#e6194b', 
              'Lime': '#d2f53c', 
              'Pink': '#fabebe', 
              'Lavender': '#e6beff', 

              'Brown': '#aa6e28', 
              'Mint': '#aaffc3', 
              'Olive': '#808000', 
              'Coral': '#ffd8b1',  
              'Grey': '#808080', 
#               'Lavender': '#e6beff', 
             }
colornames = list(color_dict.keys())

In [100]:
def aggregate_plots(fig_dir_list):
    col_n = 8

    images = [Image.open(x) for x in fig_dir_list]
    widths, heights = zip(*(i.size for i in images))

    total_width = max(widths)*col_n
    total_height = max(heights)*math.ceil(len(images)/col_n)
    # max_height = max(heights)

    # new_im = Image.new('RGB', (total_width, max_height))
    fig_agg = Image.new('RGB', (total_width, total_height), (255, 255, 255, 0))

    x_offset = 0
    for i, im in enumerate(images):
        x_offset = i%col_n
        y_offset = math.floor(i/col_n)
        fig_agg.paste(im, (x_offset*im.size[0],y_offset*im.size[1]))
        
    fig_agg.save(fig_dir_list[0].split('scatter')[0]+'HPsearch_agg.png')


In [101]:
# optimal_metric = 'F1'
optimal_metric = 'loss'
display_metric = ['F1']

def plot_metrics(task_name, metric_names, key, dict_task, variable_name, training_params_list, outputdir, plt_optimal=False):
    fontsize_label = {
        'subtitle': 20,
        'axtitle': 17,
        'xytitle': 17,
        'annotate':6,
    }
    
    dann_color = 'Blue'
    tgt_color = 'Green'
    
    dict_task_HP = dict_task[key]
    variable_value = next(training_params for training_params in training_params_list if training_params['HP_name'] == key)[variable_name]
    
    fig = plt.figure(figsize=(len(metric_names)*5, 5), dpi=100+len(metric_names)*5)
    fig.suptitle('{}\n({}={})'.format(task_name,variable_name,variable_value), fontsize=fontsize_label['subtitle'], y=1.12)

    dann_mean_optimal_metric = dict_task_HP['performance_dann'][optimal_metric].values.mean()

    for i, metric_name in enumerate(metric_names):
        source_dpt = dict_task_HP['performance_source'][metric_name].values
        dann_dpt = dict_task_HP['performance_dann'][metric_name].values
#         display(dict_task_HP['performance_dann'])
#         sys.exit()
        target_dpt = dict_task_HP['performance_target'][metric_name].values

        ax = fig.add_subplot(1, len(metric_names), i+1)
        ax.scatter(source_dpt, dann_dpt, s=40, marker='o', alpha=0.5, c=color_dict[dann_color])
#         ax.set_xlabel('source({:.4f}±{:.4f})'.format(np.nanmean(source_dpt),np.nanstd(source_dpt)), fontsize=fontsize_label['xytitle'])
#         ax.set_ylabel('DANN({:.4f}±{:.4f})'.format(np.nanmean(dann_dpt),np.nanstd(dann_dpt)), fontsize=fontsize_label['xytitle'], c=color_dict[dann_color])
        ax.set_xlabel('source({:.4f}±{:.4f})'.format(np.mean(source_dpt),np.std(source_dpt)), fontsize=fontsize_label['xytitle'])
        ax.set_ylabel('DANN({:.4f}±{:.4f})'.format(np.mean(dann_dpt),np.std(dann_dpt)), fontsize=fontsize_label['xytitle'], c=color_dict[dann_color])

        ax_r = ax.twinx()  # instantiate a second axes that shares the same x-axis
        ax_r.scatter(source_dpt, target_dpt, s=40, marker='o', alpha=0.5, c=color_dict[tgt_color])
#         ax_r.set_ylabel('target({:.4f}±{:.4f})'.format(np.nanmean(target_dpt),np.nanstd(target_dpt)), fontsize=fontsize_label['xytitle'], c=color_dict[tgt_color]) 
        ax_r.set_ylabel('target({:.4f}±{:.4f})'.format(np.mean(target_dpt),np.std(target_dpt)), fontsize=fontsize_label['xytitle'], c=color_dict[tgt_color]) 

        if metric_name == 'PAD':
            ax_xlim = ax.get_xlim()
            ax_ylim = ax.get_ylim()
            ax_r_xlim = ax_r.get_xlim()
            ax_r_ylim = ax_r.get_ylim()
            x_min = min(ax_xlim+ax_r_xlim)
            x_max = max(ax_xlim+ax_r_xlim)
            y_min = min(ax_ylim+ax_r_ylim)
            y_max = max(ax_ylim+ax_r_ylim)
            ax.plot([x_min, x_max], [y_min, y_max], c=".3", linewidth=1, alpha=0.4)
            ax.set_xlim(x_min,x_max)
            ax.set_ylim(y_min,y_max)
            ax_r.set_ylim(y_min,y_max)

        else:
            ax.plot([0, 1], [0, 1], c=".3", linewidth=1, alpha=0.4)
            ax.set_xlim(0,1)
            ax.set_ylim(0,1)
            ax_r.set_ylim(0,1)
        
        for i in range(source_dpt.shape[0]):
            ax.annotate(i, (source_dpt[i], dann_dpt[i]),alpha=0.9,fontsize=fontsize_label['annotate'], c=color_dict[dann_color])
            ax_r.annotate(i, (source_dpt[i], target_dpt[i]),alpha=0.9,fontsize=fontsize_label['annotate'], c=color_dict[tgt_color])

#         ax.set_title('{}({:+.4f} / {:+.4f})'.format(metric_name,np.nanmean(dann_dpt)-np.nanmean(source_dpt),np.nanmean(target_dpt)-np.nanmean(source_dpt)), fontsize=fontsize_label['axtitle'])
        ax.set_title('{}({:+.4f} / {:+.4f})'.format(metric_name,np.mean(dann_dpt)-np.mean(source_dpt),np.mean(target_dpt)-np.mean(source_dpt)), fontsize=fontsize_label['axtitle'])


    fig.tight_layout()
    fig_folder = outputdir+task_name
    if not os.path.exists(fig_folder):
        os.makedirs(fig_folder)
    if plt_optimal:
        fig_dir = fig_folder+'/scatter_{}_optimal.png'.format(key)
    else:
        fig_dir = fig_folder+'/scatter_{}.png'.format(key)

    fig.savefig(fig_dir, bbox_inches = "tight")
    pyplot.close(fig)

    return dann_mean_optimal_metric, fig_dir


for task_name, dict_task in dict_task_all.items():
    if optimal_metric == 'loss':
        dann_mean_optimal = 100000
    elif optimal_metric == 'F1':
        dann_mean_optimal = 0
    fig_dir_list = []
    for key in dict_task.keys():
        dann_mean, fig_dir = plot_metrics(task_name, display_metric, key, dict_task, variable_name, training_params_list, outputdir)
        if (optimal_metric=='loss' and dann_mean_optimal>dann_mean) or (optimal_metric=='F1' and dann_mean_optimal<dann_mean): # find the lowest loss
            dann_mean_optimal = dann_mean
            key_optimal = key
        fig_dir_list.append(fig_dir)

    _, fig_dir = plot_metrics(task_name, display_metric, key_optimal, dict_task, variable_name, training_params_list, outputdir, plt_optimal=True)
    fig_dir_list.append(fig_dir)

    aggregate_plots(fig_dir_list)

In [102]:
def F1_checker_each(dict_task_HP):
    for key in dict_task_HP.keys():
        arr_sens = dict_task_HP[key]['sensitivity'].values
        arr_prec = dict_task_HP[key]['precision'].values
        arr_F1 = dict_task_HP[key]['F1'].values

        test_F1 = 2*(arr_sens*arr_prec)/(arr_sens+arr_prec)
        print('arr_sens:', arr_sens)
        print('arr_prec:', arr_prec)
        print('arr_F1:', arr_F1)
        print('test_F1:', test_F1)
#         sys.exit()
        if np.nansum(np.abs(test_F1-arr_F1)) > 0.00001:
            print('***      F1 for a CV, a rep computed incorrectly in {}     ***'.format(key))

In [103]:
def F1_checker_mean(dict_df):
    for column_name in dict_df['sensitivity'].columns.values:
        for row_name in dict_df['sensitivity'].index.values:
            float_sens = float(dict_df['sensitivity'].loc[row_name, column_name].split('±')[0])
            float_prec = float(dict_df['precision'].loc[row_name, column_name].split('±')[0])
            float_F1 = float(dict_df['F1'].loc[row_name, column_name].split('±')[0])

            test_F1 = 2*(float_sens*float_prec)/(float_sens+float_prec)

            if np.abs(float_F1-test_F1) > 0.00001:
                print('***      mean F1 for all CV, all rep computed incorrectly in {}     ***'.format(key))
                # # print('float_sens:', float_sens)
                # # print('float_prec:', float_prec)
                # # print('float_F1:', float_F1)
                # # print('test_F1:', test_F1)

In [104]:
# debug_F1 = True

dict_df_all = {}


for task_name, dict_task in dict_task_all.items():
    df = pd.DataFrame(columns=list(dict_task.keys()),index=['source','DANN','target'])
    dict_df = dict( zip(metric_names,[df.copy(), df.copy(), df.copy(), df.copy(), df.copy()]))

    for key in dict_task.keys():
        dict_task_HP = dict_task[key]
        
        if debug_F1:
            F1_checker_each(dict_task_HP)

        for i, metric_name in enumerate(metric_names):
            source_dpt = dict_task_HP['performance_source'][metric_name].values
            dann_dpt = dict_task_HP['performance_dann'][metric_name].values
            target_dpt = dict_task_HP['performance_target'][metric_name].values
            
            dict_df[metric_name].loc['source', key] = '{:.4f}±{:.4f}'.format(np.nanmean(source_dpt),np.nanstd(source_dpt))
            dict_df[metric_name].loc['DANN', key] = '{:.4f}±{:.4f}'.format(np.nanmean(dann_dpt),np.nanstd(dann_dpt))
            dict_df[metric_name].loc['target', key] = '{:.4f}±{:.4f}'.format(np.nanmean(target_dpt),np.nanstd(target_dpt))

    # Create a Pandas Excel writer using XlsxWriter as the engine
    df_outputdir = outputdir+task_name
    if not os.path.exists(df_outputdir):
        os.makedirs(df_outputdir)
        
    writer = pd.ExcelWriter(df_outputdir+'/allmetrics.xlsx', engine='xlsxwriter')

    for i, metric_name in enumerate(metric_names):
        print(task_name, metric_name)
        display(dict_df[metric_name])
        dict_df[metric_name].to_excel(writer, sheet_name=metric_name)
        
        if debug_F1:
            F1_checker_mean(dict_df)

    writer.save()

    dict_df_all[task_name] = dict_df.copy()
    


arr_sens: [0.97560976 0.92307692 0.81481481 0.96428571 1.         0.94444444
 1.         0.97142857 1.         0.95081967 1.         0.89534884
 0.88888889 1.         1.         1.         1.         1.
 1.         0.88888889 0.93939394 0.91935484 1.         0.92592593
 1.        ]
arr_prec: [1.         1.         1.         1.         1.         1.
 0.96774194 0.97142857 1.         1.         0.96       0.97468354
 1.         0.92       0.95238095 1.         0.97222222 0.98550725
 1.         0.94117647 1.         1.         1.         0.96153846
 0.97222222]
arr_F1: [0.98765432 0.96       0.89795918 0.98181818 1.         0.97142857
 0.98360656 0.97142857 1.         0.97478992 0.97959184 0.93333333
 0.94117647 0.95833333 0.97560976 1.         0.98591549 0.99270073
 1.         0.91428571 0.96875    0.95798319 1.         0.94339623
 0.98591549]
test_F1: [0.98765432 0.96       0.89795918 0.98181818 1.         0.97142857
 0.98360656 0.97142857 1.         0.97478992 0.97959184 0.93333333
 0

KeyError: 'specificity'

In [105]:
dict_task_HP['performance_source']

,acc,sensitivity,precision,F1,PAD,loss
0,0.991379,0.975610,1.000000,0.987654,1.936674,0.000402
1,0.965517,0.923077,1.000000,0.960000,1.925210,0.000692
2,0.929577,0.814815,1.000000,0.897959,1.968897,0.001946
3,0.983193,0.964286,1.000000,0.981818,1.988645,0.000104
4,1.000000,1.000000,1.000000,1.000000,1.999840,0.000599
5,0.979167,0.944444,1.000000,0.971429,1.968978,0.001018
6,0.987805,1.000000,0.967742,0.983607,1.732203,0.000783
7,0.977011,0.971429,0.971429,0.971429,1.974279,0.001638
8,1.000000,1.000000,1.000000,1.000000,1.950928,0.000461
9,0.978261,0.950820,1.000000,0.974790,1.960762,0.000087


In [ ]:
dict_df.keys()

In [ ]:
row_names = dict_df_all[task_name]['acc'].index

In [ ]:
df_paper_tables = {}

for key in dict_df.keys():
    if key == 'PAD':
        continue
            
    df_task_agg = pd.DataFrame(index=row_names)

    for task_name, dict_task in dict_task_all.items():
        dict_df = dict_df_all[task_name]

        dict_df_key_task = dict_df[key].rename(columns={'HP_fixed': task_name})
        df_task_agg = df_task_agg.join(dict_df_key_task, how='outer')

    df_paper_tables[key] = df_task_agg

    sys.exit()


In [90]:
df_paper_tables['acc'].to_csv ('export_dataframe.csv', header=True)


In [88]:
df_paper_tables['acc']

,UMAFall_chest_UMAFall_waist,UMAFall_chest_UMAFall_wrist,UMAFall_chest_UMAFall_ankle,UMAFall_waist_UMAFall_wrist,UMAFall_waist_UMAFall_ankle,UMAFall_waist_UMAFall_chest,UMAFall_wrist_UMAFall_ankle,UMAFall_wrist_UMAFall_waist,UMAFall_wrist_UMAFall_chest,UMAFall_ankle_UMAFall_chest,UMAFall_ankle_UMAFall_waist,UMAFall_ankle_UMAFall_wrist
source,0.9764±0.0207,0.7548±0.1223,0.9023±0.0496,0.6932±0.1257,0.8341±0.1470,0.9509±0.0566,0.7957±0.0874,0.8090±0.0998,0.7462±0.0757,0.8722±0.1071,0.8523±0.1025,0.6704±0.1132
DANN,0.9614±0.0587,0.8998±0.0776,0.8671±0.0668,0.7989±0.1223,0.8670±0.0665,0.9449±0.0550,0.8277±0.0722,0.8025±0.1027,0.8090±0.0810,0.8854±0.0755,0.9108±0.0799,0.7131±0.1356
target,0.9852±0.0225,0.9575±0.0314,0.9409±0.0517,0.9453±0.0383,0.8333±0.1959,0.9587±0.0470,0.9018±0.0648,0.9847±0.0275,0.9684±0.0308,0.9238±0.0779,0.9486±0.0446,0.9652±0.0230


In [42]:
# df_empty = pd.DataFrame(columns = column_names)
df_empty = pd.DataFrame(columns = list(dict_task_all.keys()))
df_empty

,UMAFall_chest_UMAFall_wrist,UMAFall_chest_UMAFall_waist,UMAFall_chest_UMAFall_ankle,UMAFall_wrist_UMAFall_chest,UMAFall_wrist_UMAFall_waist,UMAFall_wrist_UMAFall_ankle,UMAFall_waist_UMAFall_chest,UMAFall_waist_UMAFall_wrist,UMAFall_waist_UMAFall_ankle,UMAFall_ankle_UMAFall_chest,UMAFall_ankle_UMAFall_wrist,UMAFall_ankle_UMAFall_waist


In [53]:
df_empty = pd.DataFrame(index=dict_df['acc'].index)
df_empty

""
source
DANN
target


In [55]:
aaa = dict_df['key'].rename(columns={'HP_fixed': 'UMAFall_chest_UMAFall_wrist'})
# bbb = dict_df['acc'].rename(columns={'HP_fixed': 'UMAFall_chest_UMAFall_wrist2'})

df_empty = df_empty.join(aaa, how='outer')
df_empty

ValueError: columns overlap but no suffix specified: Index(['UMAFall_chest_UMAFall_wrist'], dtype='object')

In [38]:
df_empty = df_empty['acc'].append(dict_df['acc'].values)

# df_empty = df_empty['bcc'].append(dict_df['acc'].values)
# df_empty.append(dict_df['acc'])

KeyError: 'acc'

,HP_fixed
source,0.7548±0.1223
DANN,0.8998±0.0776
target,0.9575±0.0314


In [22]:
dict_df_all['UMAFall_chest_UMAFall_wrist']['acc']

,HP_fixed
source,0.7548±0.1223
DANN,0.8998±0.0776
target,0.9575±0.0314


In [ ]:
for 